In [61]:
import os
import pandas as pd
import numpy as np
import matplotlib
from  matplotlib import pyplot as plt
%matplotlib qt
import scipy
import librosa
import librosa.display
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import FVA.FVA as fva
from FVA.lpc import *
from FVA.detect import *
from FVA.mfcc import *
from FVA.animation import *
from FVAmanager import *
from DB.DBmanager import *

In [79]:
bias  = "FVA/target_sounds/"
name  = "recorded.wav"
FM  =   FVAmanager(bias+name)
fm  =   FM.FR_formants()
fp  =   FM.FR_pitch()
print(fm,fp,sep='\n')
dellist = []
for i,x in enumerate(fm):
    if x < fp + 50 and x > fp - 50: dellist.append(i)
for i in dellist:
    del fm[i]

7/7 [==============================] - 2s 211ms/step
CREPE: Saved the estimated frequencies and confidence values at FVA/target_sounds/recorded.f0.csv
[556.9694911067194, 1692.697010869565, 2809.3595602766795, 3430.3328804347825, 4122.118947628458, 4522.483325098814, 5151.627346837944, 5519.308917984189, 6046.319169960474, 6577.414772727272, 7040.421195652173, 7404.017416007905, 7620.541007905138, 9663.216403162054, 10583.782114624506]
nan


In [80]:
print(fm,fp,sep='\n')

[556.9694911067194, 1692.697010869565, 2809.3595602766795, 3430.3328804347825, 4122.118947628458, 4522.483325098814, 5151.627346837944, 5519.308917984189, 6046.319169960474, 6577.414772727272, 7040.421195652173, 7404.017416007905, 7620.541007905138, 9663.216403162054, 10583.782114624506]
nan


In [62]:
result, users   =   getDB()
print(result)
print(users)

                         id    audio answer                        at  \
0      007jtO0zboMfmk6WmEbV  2_1.wav      ㅗ  2023-09-19T14:04:42.169Z   
1      00GdM1CpptAw1UttIfYg  7_7.wav      ㅐ  2023-09-26T12:25:47.279Z   
2      00oWxK2Sig2FQ4dAHbO9  1_3.wav      ㅜ  2023-09-26T12:30:59.025Z   
3      01bEPzEm2zvvQ5bXVEA6  8_4.wav      ㅏ  2023-09-28T14:15:13.334Z   
4      01lyhssv0mmHv4aoOdnp  4_2.wav      ㅗ  2023-09-26T13:02:26.094Z   
...                     ...      ...    ...                       ...   
10227  zxeEOvxplVuy1cJwcJQZ  9_8.wav      ㅔ  2023-09-18T14:04:11.991Z   
10228  zxeSqLYlV4lHLIp9iozg  8_5.wav      ㅏ  2023-09-27T14:29:59.902Z   
10229  zyx8FzyY6CZgnkbD4fL2  4_4.wav      ㅓ  2023-09-11T16:47:22.943Z   
10230  zzlrP5A3adabEEpoz3xZ  6_5.wav      ㅐ  2023-09-27T12:57:15.051Z   
10231  zzs3vTc0mOkqH7pSjiC6  6_4.wav      ㅗ  2023-09-27T01:31:49.926Z   

                               user  
0      dznwXCu4dMaMrSrKWoUy1QTLQht1  
1      Do457fhfRKex3AtI5dQ8O2U7WEJ3  
2      95

In [63]:
formants = pd.read_excel("DB/sounds.xlsx")

In [64]:
vowels = {'ㅏ':0,'ㅓ':1,'ㅗ':2,'ㅜ':3,'ㅡ':4,'ㅣ':5,'ㅐ':6,'ㅔ':7}
counts = np.zeros((8,11,11))

In [65]:
for i in range(result.shape[0]):
  counts[vowels[result['answer'][i]]][int(result['audio'][i].split('_')[0])][int(result['audio'][i].split('_')[1].split('.')[0])]+=1

In [81]:
F1 = [float(formants[i+1][0]) for i in range(10)]
F2 = [float(formants[i+1][1]) for i in range(10)]
logF1 = np.log10(np.array(F1))
logF2 = np.log10(np.array(F2))
nwF1 = np.log10(np.array([fm[0]]))[0]
nwF2 = np.log10(np.array([fm[1]]))[0]
nwF1in  = -1
nwF1v   = 987654321.
nwF2in  = -1
nwF2v   = 987654321.
for i in range(10):
  if nwF1v> abs(logF1[i]-nwF1):
    nwF1v = abs(logF1[i]-nwF1)
    nwF1in= i
  if nwF2v> abs(logF2[i]-nwF2):
    nwF2v = abs(logF2[i]-nwF2)
    nwF2in= i
print(F1[nwF1in],F2[nwF2in])

540.9848448436562 1829.0708847622484


In [82]:
tmp = counts.transpose(1, 2, 0)
print({v:k for k,v in vowels.items()}[tmp[nwF1in+1][nwF2in+1].argmax()])

ㅔ


In [66]:
def softmax(x): # numpy array!!
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def probability(a):
    sum = a.sum()
    return a/sum

In [83]:
tmp[nwF1in+1][nwF2in+1]

array([ 0.,  0.,  0.,  1., 12.,  0., 36., 42.])

In [84]:
a = np.array(tmp[nwF1in+1][nwF2in+1])
b = probability(a)
a = softmax(a)

precision = 1
pairs_a = [[round(a[i]*100,precision),list(vowels.keys())[i]] for i in range(len(vowels.items()))]
pairs_b = [[round(b[i]*100,precision),list(vowels.keys())[i]] for i in range(len(vowels.items()))]
pairs_b = sorted(pairs_b,key=lambda x:x[0],reverse=True)

nVW = 4
for x in pairs_b[:nVW]:
    print(x[1],": ",x[0],"%",sep='',end='\n')
    
print(f"Pitch : {round(fp,1)}Hz")

print(f'Formants : ',end='')
nFM = 4
for i,x in enumerate(fm[:nFM]):
    print(round(x,1),"Hz",sep='',end='\n' if i+1==nFM else ', ')

ㅔ: 46.2%
ㅐ: 39.6%
ㅡ: 13.2%
ㅜ: 1.1%
Pitch : nanHz
Formants : 557.0Hz, 1692.7Hz, 2809.4Hz, 3430.3Hz
